<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/FinalModule_Coursera/images/IDSNlogo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>

<h1 align="center"><font size="5">Classification with Python</font></h1>


In this notebook we try to practice all the classification algorithms that we have learned in this course.

We load a dataset using Pandas library, and apply the following algorithms, and find the best one for this specific dataset by accuracy evaluation methods.

Let's first load required libraries:


In [1]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

### About dataset


This dataset is about past loans. The **Loan_train.csv** data set includes details of 346 customers whose loan are already paid off or defaulted. It includes following fields:

| Field          | Description                                                                           |
| -------------- | ------------------------------------------------------------------------------------- |
| Loan_status    | Whether a loan is paid off on in collection                                           |
| Principal      | Basic principal loan amount at the                                                    |
| Terms          | Origination terms which can be weekly (7 days), biweekly, and monthly payoff schedule |
| Effective_date | When the loan got originated and took effects                                         |
| Due_date       | Since it’s one-time payoff schedule, each loan has one single due date                |
| Age            | Age of applicant                                                                      |
| Education      | Education of applicant                                                                |
| Gender         | The gender of applicant                                                               |


Let's download the dataset


In [2]:
!wget -O loan_train.csv https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/FinalModule_Coursera/data/loan_train.csv

'wget' is not recognized as an internal or external command,
operable program or batch file.


### Load Data From CSV File


In [3]:
df = pd.read_csv('loan_train.csv')
df.head()

FileNotFoundError: [Errno 2] File b'loan_train.csv' does not exist: b'loan_train.csv'

In [ ]:
df.shape

### Convert to date time object


In [ ]:
df['due_date'] = pd.to_datetime(df['due_date'])
df['effective_date'] = pd.to_datetime(df['effective_date'])
df.head()

# Data visualization and pre-processing


Let’s see how many of each class is in our data set


In [ ]:
df['loan_status'].value_counts()

260 people have paid off the loan on time while 86 have gone into collection


Let's plot some columns to underestand data better:


In [ ]:
# notice: installing seaborn might takes a few minutes
!conda install -c anaconda seaborn -y

In [ ]:
import seaborn as sns

bins = np.linspace(df.Principal.min(), df.Principal.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'Principal', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

In [ ]:
bins = np.linspace(df.age.min(), df.age.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'age', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

# Pre-processing:  Feature selection/extraction


### Let's look at the day of the week people get the loan


In [ ]:
df['dayofweek'] = df['effective_date'].dt.dayofweek
bins = np.linspace(df.dayofweek.min(), df.dayofweek.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'dayofweek', bins=bins, ec="k")
g.axes[-1].legend()
plt.show()


We see that people who get the loan at the end of the week don't pay it off, so let's use Feature binarization to set a threshold value less than day 4


In [ ]:
df['weekend'] = df['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)
df.head()

## Convert Categorical features to numerical values


Let's look at gender:


In [ ]:
df.groupby(['Gender'])['loan_status'].value_counts(normalize=True)

86 % of female pay there loans while only 73 % of males pay there loan


Let's convert male to 0 and female to 1:


In [ ]:
df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
df.head()

## One Hot Encoding

#### How about education?


In [ ]:
df.groupby(['education'])['loan_status'].value_counts(normalize=True)

#### Features before One Hot Encoding


In [ ]:
df[['Principal','terms','age','Gender','education']].head()

#### Use one hot encoding technique to conver categorical varables to binary variables and append them to the feature Data Frame


In [ ]:
Feature = df[['Principal','terms','age','Gender','weekend']]
Feature = pd.concat([Feature,pd.get_dummies(df['education'])], axis=1)
Feature.drop(['Master or Above'], axis = 1,inplace=True)
Feature.head()


### Feature Selection


Let's define feature sets, X:


In [ ]:
X = Feature
X[0:5]

What are our lables?


In [ ]:
y = df['loan_status'].values
y[0:5]

## Normalize Data


Data Standardization give data zero mean and unit variance (technically should be done after train test split)


In [ ]:
X= preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

# Classification


Now, it is your turn, use the training set to build an accurate model. Then use the test set to report the accuracy of the model
You should use the following algorithm:

*   K Nearest Neighbor(KNN)
*   Decision Tree
*   Support Vector Machine
*   Logistic Regression

\__ Notice:\__

*   You can go above and change the pre-processing, feature selection, feature-extraction, and so on, to make a better model.
*   You should use either scikit-learn, Scipy or Numpy libraries for developing the classification algorithms.
*   You should include the code of the algorithm in the following cells.


# K Nearest Neighbor(KNN)

Notice: You should find the best k to build the model with the best accuracy.\
**warning:** You should not use the **loan_test.csv** for finding the best k, however, you can split your train_loan.csv into train and test to find the best **k**.


In [ ]:
#Train-Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

#Import Classifier
from sklearn.neighbors import KNeighborsClassifier

#Training
#Let's start the algorithm with k=4:
k = 4
#Train Model and Predict  
neigh4 = KNeighborsClassifier(n_neighbors = k).fit(X_train,y_train)
neigh4

#Predicting
#We can use the model to make predictions on the test set:
yhat4 = neigh4.predict(X_test)
yhat4[0:5]

#Accuracy Evaluation
from sklearn import metrics
print("Train set Accuracy for k=4: ", metrics.accuracy_score(y_train, neigh4.predict(X_train)))
print("Test set Accuracy for k=4: ", metrics.accuracy_score(y_test, yhat4))

#Looping 10 times to find the best k
#We can calculate the accuracy of KNN for different values of k.
Ks = 10
mean_acc = np.zeros((Ks-1))
std_acc = np.zeros((Ks-1))

for n in range(1,Ks):
    
    #Train Model and Predict  
    neigh = KNeighborsClassifier(n_neighbors = n).fit(X_train,y_train)
    yhat=neigh.predict(X_test)
    mean_acc[n-1] = metrics.accuracy_score(y_test, yhat)

    
    std_acc[n-1]=np.std(yhat==y_test)/np.sqrt(yhat.shape[0])

mean_acc

#Plotting the multible values of k
#Plot the model accuracy for a different number of neighbors.
plt.plot(range(1,Ks),mean_acc,'g')
plt.fill_between(range(1,Ks),mean_acc - 1 * std_acc,mean_acc + 1 * std_acc, alpha=0.10)
plt.fill_between(range(1,Ks),mean_acc - 3 * std_acc,mean_acc + 3 * std_acc, alpha=0.10,color="green")
plt.legend(('Accuracy ', '+/- 1xstd','+/- 3xstd'))
plt.ylabel('Accuracy ')
plt.xlabel('Number of Neighbors (K)')
plt.tight_layout()
plt.show()

#Print the Result of the best k
print( "The best accuracy was with", mean_acc.max(), "with k=", mean_acc.argmax()+1) 




In [ ]:
#Use the best accuracy k value of 7:
#Train Model and Predict  
neigh7 = KNeighborsClassifier(n_neighbors = 7).fit(X_train,y_train)
neigh7

#Predicting
#We can use the model to make predictions on the test set:
yhat7 = neigh7.predict(X_test)
yhat7[0:5]

#Accuracy Evaluation when k=7
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import jaccard_score
print("Train set KNN Jaccard Accuracy: ", metrics.accuracy_score(y_train, neigh7.predict(X_train)))
print("Test set KNN accard Accuracy: ", metrics.accuracy_score(y_test, yhat7))
print("Train set KNN F1 Accuracy" % f1_score(y_train, neigh7.predict(X_train), average='weighted'))
print("Test set KNN F1 Accuracy" % f1_score(y_test, neigh7.predict(X_test), average='weighted'))


DECISION TREE

In [ ]:
#Import Classifier
from sklearn.tree import DecisionTreeClassifier

#Train-Test Split
from sklearn.model_selection import train_test_split
X_trainset, X_testset, y_trainset, y_testset = train_test_split(X, y, test_size=0.3, random_state=3)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

#Training
#Must create an instance of the DecisionTreeClassifier, called loanTree
loanTree = DecisionTreeClassifier(criterion="entropy", max_depth = 4)
loanTree
#train with train-test data
loanTree.fit(X_trainset, y_trainset)

#Predicting
predTree = loanTree.predict(X_testset)
#Print out predicted and actual values
print(predTree[0:5])
print(y_testset[0:5])





In [ ]:
#Accuracy Evaluation
#Need to import matplotlib.pyplot
import matplotlib.pyplot as plt
print("Train set Decision Trees's Jaccard Accuracy: ", metrics.accuracy_score(y_traintset, predTree))
print("Test set Decision Trees's Jaccard Accuracy: ", metrics.accuracy_score(y_testset, predTree))
print("Train set Decision Tree's F1 Accuracy:" , f1_score(y_trainset, predTree), average='weighted'))
print("Test set Decision Tree's F1 Accuracy:" , f1_score(y_testset, predTree), average='weighted'))



In [ ]:
#Visualize the Tree
#May need to install the following:
#!conda install -c conda-forge pydotplus -y
#!conda install -c conda-forge python-graphviz -y
#Import from sklearn
import sklearn.tree as tree

tree.plot_tree(loanTree)
plt.show()


# Support Vector Machine


In [ ]:
#Train-Test Split
#Is already completed

#Import Classifier
from sklearn import svm
#Choose a kernel function as linear, polynomial, radial basis function(RBF), Sigmoid
svm_clf=svm.SVC(kernel='rbf')

#Training
svm_clf.fit(X_train, y_train)

#Predicting
yhatSVM=svm_clf.predict(X_test)
yhatSVM[0:5]

#Accuracy Evaluation
from sklearn.metrics import classification_report, confusion_matrix
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')



# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, yhatSVM, labels=['PAIDOFF', 'COLLECTION'])
np.set_printoptions(precision=2)

print (classification_report(y_test, yhatSVM))



In [ ]:
# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['PAIDOFF','COLLECTION'],normalize= False,  title='Confusion matrix')

In [ ]:
#Determine Accuracy Evaluation of Jaccard and F1 scores 
print("Train set SVM Jaccard Accuracy: ", metrics.accuracy_score(y_train, yhatSVM))
print("Test set SVM Jaccard Accuracy: ", metrics.accuracy_score(y_test, yhatSVM))
print("Train set SVM F1 Accuracy:" , f1_score(y_train, yhatSVM, average='weighted'))
print("Test set SVM F1 Accuracy:" , f1_score(y_test, yhatSVM, average='weighted'))


# Logistic Regression


In [ ]:
#Preprocessing of the data - target data (y) must be an integer
df['loan_status'].replace(to_replace=('PAIDOFF', 'COLLECTION'), value=[0,1], inplace=True)
df.head()

#Note: X and y have already been identified,normalized, 
#and made into a train & test set from prior application

#Import Classifier
from sklearn.linear_model import LogisticRegression

#Training
#Choose optimizer solver: 'newton-cg', 'liblinear', 'sag', 'saga', 'lbfgs'
lr = LogisticRegression(C=0.01, solver='liblinear').fit(X_train,y_train)
lr

#Predicting
yhatLR = lr.predict(X_test)
yhatLR

#Perform predict proba estimates
yhatLR_prob = lr.predict_proba(X_test)
yhatLR_prob

#Accuracy Evaluation including log loss
from sklearn.metrics import log_loss
print("Train set LR Jaccard Accuracy: ", metrics.accuracy_score(y_train, yhatLR))
print("Test set LR Jaccard Accuracy: ", metrics.accuracy_score(y_test, yhatLR))
print("Train set LR F1 Accuracy:" , f1_score(y_train, yhatLR, average='weighted'))
print("Test set LR F1 Accuracy:" , f1_score(y_test, yhatLR, average='weighted'))
print("Train set LR Log_Loss:", log_loss(y_train, yhatLR_prob)) 
print("Test set LR Log_Loss:", log_loss(y_test, yhatLR_prob))


# Model Evaluation using Test set


In [ ]:
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss


First, download and load the test set:


In [ ]:
!wget -O loan_test.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_test.csv

### Load Test set for evaluation


In [ ]:
test_df = pd.read_csv('loan_test.csv')
test_df.head()

In [ ]:
#Convert to date time object
test_df['due_date'] = pd.to_datetime(test_df['due_date'])
test_df['effective_date'] = pd.to_datetime(test_df['effective_date'])
test_df.head()

In [ ]:
#use Feature binarization to set a threshold value less than day 4 of week
test_df['dayofweek'] = test_df['effective_date'].dt.dayofweek
bins = np.linspace(test_df.dayofweek.min(), test_df.dayofweek.max(), 10)


test_df['weekend'] = test_df['dayofweek'].apply(lambda x: 1 if (x>3) else 0)
test_df.head()

In [ ]:
#Convert category to numeric: male to 0 and female to 1
test_df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
test_df.head()

In [ ]:
#Convert category to numeric: PAIDOFF to 0 and COLLECTION to 1
test_df['loan_status'].replace(to_replace=['PAIDOFF','COLLECTION'], value=[0,1],inplace=True)
test_df.head()

In [ ]:
#Use one hot encoding to convert category to binary
FeatureX = test_df[['Principal','terms','age','Gender','weekend']]
FeatureX = pd.concat([FeatureX,pd.get_dummies(test_df['education'])], axis=1)
FeatureX.drop(['Master or Above'], axis = 1,inplace=True)
FeatureX.head()


In [ ]:
#Define X
X_ltest= FeatureX
X_ltest[0:5]


In [ ]:
#Define our Labels 0=PAIDOFF, 1=COLLECTION
y_ltest= test_df['loan_status'].values
y_ltest[0:5]

In [ ]:
#Normalize the data
X_ltest = preprocessing.StandardScaler().fit(X).transform(X)
X_ltest[0:5]

In [ ]:
#Predict KNN using loan_test dataset & get metrics
pred_knn = neigh7.predict(X_ltest)
#KNN Metrics
knn_jaccard = jaccard_score(y_ltest, pred_knn)
knn_f1 = f1_score(y_ltest, pred_knn, average='weighted')

#Predict DecisionTree using loan_test dataset & get metrics
pred_dt = loanTree.predict(X_ltest)
#Decision Tree Metrics
dt_jaccard = jaccard_score(y_ltest, pred_dt)
dt_f1 = f1_score(y_ltest, pred_dt, average='weighted')

#Predict SVM using loan_test dataset & get metrics
pred_svm = svm_clf.predict(X_ltest)
#SVM Metrics
svm_jaccard = jaccard_score(y_ltest, pred_svm)
svm_f1 = f1_score(y_ltest, pred_svm, average='weighted')

#Predict LRusing loan_test dataset & get metrics
pred_lr = lr.predict(X_ltest)
#LR Metrics
lr_jaccard = jaccard_score(y_ltest, pred_lr)
lr_f1 = f1_score(y_ltest, pred_lr, average='weighted')
#Compute proba to also determine log loss
pred_lrproba = lr.predict_proba(X_ltest)
lr_logloss = log_loss(y_ltest, pred_lrproba)


print("                     Jaccard            F1-Score                      Logloss      ")
print("KNN                " knn_jaccard"        " knn_f1 "                       N/A")
print("Decision Tree      " dt_jaccard "        " dt_f1 "                        N/A")
print("SVM                " svm_jaccard"        " svm_f1 "                       N/A")
print("Logistic Regression" lr_jaccard "        " lr_f1 "                   " lr_logloss)

# Report

You should be able to report the accuracy of the built model using different evaluation metrics:


| Algorithm          | Jaccard | F1-score | LogLoss |
| ------------------ | ------- | -------- | ------- |
| KNN                | ?       | ?        | NA      |
| Decision Tree      | ?       | ?        | NA      |
| SVM                | ?       | ?        | NA      |
| LogisticRegression | ?       | ?        | ?       |


<h2>Want to learn more?</h2>

IBM SPSS Modeler is a comprehensive analytics platform that has many machine learning algorithms. It has been designed to bring predictive intelligence to decisions made by individuals, by groups, by systems – by your enterprise as a whole. A free trial is available through this course, available here: <a href="http://cocl.us/ML0101EN-SPSSModeler?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork20718538-2022-01-01">SPSS Modeler</a>

Also, you can use Watson Studio to run these notebooks faster with bigger datasets. Watson Studio is IBM's leading cloud solution for data scientists, built by data scientists. With Jupyter notebooks, RStudio, Apache Spark and popular libraries pre-packaged in the cloud, Watson Studio enables data scientists to collaborate on their projects without having to install anything. Join the fast-growing community of Watson Studio users today with a free account at <a href="https://cocl.us/ML0101EN_DSX?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork20718538-2022-01-01">Watson Studio</a>

<h3>Thanks for completing this lesson!</h3>

<h4>Author:  <a href="https://ca.linkedin.com/in/saeedaghabozorgi?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork20718538-2022-01-01?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork20718538-2022-01-01">Saeed Aghabozorgi</a></h4>
<p><a href="https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a>, PhD is a Data Scientist in IBM with a track record of developing enterprise level applications that substantially increases clients’ ability to turn data into actionable knowledge. He is a researcher in data mining field and expert in developing advanced analytic methods like machine learning and statistical modelling on large datasets.</p>

<hr>

## Change Log

| Date (YYYY-MM-DD) | Version | Changed By    | Change Description                                                             |
| ----------------- | ------- | ------------- | ------------------------------------------------------------------------------ |
| 2020-10-27        | 2.1     | Lakshmi Holla | Made changes in import statement due to updates in version of  sklearn library |
| 2020-08-27        | 2.0     | Malika Singla | Added lab to GitLab                                                            |

<hr>

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>

<p>
